In [167]:
import folium
import json
import requests
from datetime import datetime
import asyncio
import os
import pandas as pd
from census import Census
from us import states
import cenpy
import geopandas as gpd
import geojson
import numpy as np

## Getting and Parsing Jsons from retrieved from TravelTime Api in separate files
I have to get it from a separate file so I can separate each requests

TODO: Make a function that lets you choose the stations you want to get the isochrones from instead of hard coded stations

In [168]:
# list all geojson files in blue_line folder that contain the word 'walking'
paths_walking = [os.path.join('blue_line', fn) for fn in next(os.walk('blue_line'))[2] if 'walking' in fn]


stations_walking = []
# save JSON to variable
for i in range(len(paths_walking)):
    with open(paths_walking[i]) as f:
        data = json.load(f)
        stations_walking.append(data)

# repeat for bike
paths_bike = [os.path.join('blue_line', fn) for fn in next(os.walk('blue_line'))[2] if 'cycling' in fn]

stations_bike = []
for i in range(len(paths_bike)):
    with open(paths_bike[i]) as f:
        data = json.load(f)
        stations_bike.append(data)

## Defining Function To Union GeoJsons as TravelTime only allows unions up to 10 at a time

In [169]:
import shapely.geometry as sg
import geojson

# list of the biking geojsons geometries
def union_stations(stations):
    poly = sg.Polygon()
    for station in stations:
        for i in range(len(station['features'])):
            if "last station: " in station['features'][i]['properties']['search_id']:
                poly = poly.union(sg.shape(station['features'][i]['geometry']))


    # Convert the unioned polygon into a GeoJSON Feature
    union_feature = geojson.Feature(geometry=poly, properties={})

    # Create a FeatureCollection with just the union feature
    union_feature_collection = geojson.FeatureCollection([union_feature])

    # Convert the FeatureCollection to a GeoJSON string
    union_geojson_str = geojson.dumps(union_feature_collection)

    return union_geojson_str

## Getting US Census API data for population/inhabitants in each isochrone

In [177]:
def get_census_data(state_code, county_code, API_KEY):
    """
    Pull the census data for a given state and county and return a dataframe
    """
    url = f'https://api.census.gov/data/2020/dec/pl?get=NAME,H1_001N&for=block:*&in=state:{state_code}&in=county:{county_code}&key={API_KEY}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

def get_density_by_bg(df, gdf, transport_type_stations, county):
    # Split the "Block Group" from the NAME column into its own column
    df['Block Group'] = df['NAME'].str.split(',', expand=True)[1]

    # Convert the population column to an integer
    df['H1_001N'] = df['H1_001N'].astype(int)

    # groupby tract then block group and sum the population
    df = df.groupby(['state', 'county', 'tract', 'Block Group'])['H1_001N'].sum().reset_index()

    # remove the last row
    df = df[:-1]
    tracts = df['tract'].unique()

    df['Block Group'] = df['Block Group'].str.replace('Block Group ', '')
    df['H1_001N'] = df['H1_001N'].astype(str)

    # ----------------------------
    path = f'tracts_geo_json/bg/bg_{county}.geojson'

    # get all tracts in df that are in tracts
    temp = gdf[(gdf['TRACTCE'].isin(tracts)) & (gdf['COUNTYFP'] == '073')].sort_values(by='TRACTCE')

    # add population to geojson
    temp['H1_001N'] = df['H1_001N'].values

    # join Tract and Block Group to create a unique identifier
    temp['tract'] = temp['TRACTCE'] + temp['BLKGRPCE']

    # get all polygons that intersect with the union of the biking geojsons
    temp = temp[temp.intersects(sg.shape(geojson.loads(union_stations(transport_type_stations))[0]['geometry']))]

    temp.to_file(path, driver='GeoJSON')

    # ----------------------------

    data_df = pd.DataFrame()
    data_df['GEOID'] = temp['GEOID'].values
    data_df['H1_001N'] = temp['H1_001N'].values

    # convert to int
    data_df['H1_001N'] = data_df['H1_001N'].astype(int)
    # log the population
    data_df['H1_001N'] = data_df['H1_001N'].apply(lambda x: np.log(x))

    return data_df

In [178]:
# Read in shapefile
shp_file = r'CA_shape_file\bg\cb_2021_06_bg_500k.shp'
gdf = gpd.read_file(shp_file)

In [180]:
# add bike choropleth for no of population
key = "1e1f8670c9944ee8269a8d7065315bcd2f3eb0c8"
state_code = "06"
county_code = "073"

census_data = get_census_data(state_code, county_code, key)
data_df_bike = get_density_by_bg(census_data, gdf, stations_bike, 'SD')

with open(r'tracts_geo_json\bg\bg_SD.geojson', 'r') as f:
    gs_bike = geojson.load(f)

In [181]:
census_data = get_census_data(state_code, county_code, key)
data_df_walk = get_density_by_bg(census_data, gdf, stations_walking, 'SD')

with open(r'tracts_geo_json\bg\bg_SD.geojson', 'r') as f:
    gs_walk = geojson.load(f)

In [191]:
lat = 32.86926550131975
lng = -117.21402645890504
map_obj = folium.Map(location=[lat, lng], zoom_start=13)
# using gs as the geojson create a choropleth map with the population data for bike
map_obj.add_child(
folium.Choropleth(
    geo_data=gs_bike,
    name='Bike_intersections',
    data=data_df_bike,
    columns=['GEOID', 'H1_001N'],
    key_on='feature.properties.GEOID',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population'
))

# using gs as the geojson create a choropleth map with the population data for walking
map_obj.add_child(
folium.Choropleth(
    geo_data=gs_walk,
    name='Walk_intersections',
    data=data_df_walk,
    columns=['GEOID', 'H1_001N'],
    key_on='feature.properties.GEOID',
    fill_color='PuBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population'
))


# add bike in red
fg_bike = folium.FeatureGroup(name='Bike')
bike_poly = union_stations(stations_bike)
folium.GeoJson(bike_poly, style_function=lambda x: {'color':'red'}).add_to(fg_bike)

# add walking in blue
fg_walking = folium.FeatureGroup(name='Walking')
walking_poly = union_stations(stations_walking)
folium.GeoJson(walking_poly, style_function=lambda x: {'color':'blue'}).add_to(fg_walking)

# add feature groups to map
map_obj.add_child(fg_bike)
map_obj.add_child(fg_walking)

# add layer control
folium.LayerControl().add_to(map_obj)

In [192]:
map_obj